# Linear Encoder

In [2]:
import numpy as np
import mygrad as mg
import mynn

from mygrad.nnet.initializers import he_normal
from mynn.layers.dense import dense
from mygrad.nnet.losses.margin_ranking_loss import margin_ranking_loss

from mynn.optimizers.adam import Adam

%matplotlib notebook
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [3]:
with open('resnet18_features.pkl', mode="rb") as open_file:
    resnet = pickle.load(open_file)

In [8]:
a = resnet[57870]
a = a.flatten()
a.shape

(512,)

In [ ]:
class LinearEncoder:
    def __init__(self, d_input, d_output):
        """ This initializes all of the layers in our model, and sets them
        as attributes of the model.
        
        Parameters
        ----------
        d_input : int
            The size of the inputs.
            
        d_output : int
            The size of the outputs (i.e., the reduced dimensionality).
        """
        
        self.encoder = dense(d_input, d_output, weight_initializer=he_normal)
        
    def __call__(self, x):
        '''Passes data as input to our model, performing a "forward-pass".
        
        This allows us to conveniently initialize a model `m` and then send data through it
        to be classified by calling `m(x)`.
        
        Parameters
        ----------
        x : Union[numpy.ndarray, mygrad.Tensor], shape=(M, D_full)
            A batch of data consisting of M pieces of data,
            each with a dimentionality of D_full.
            
        Returns
        -------
        mygrad.Tensor, shape=(M, D_full)
            The model's prediction for each of the M pieces of data.
        '''
        
        return self.encoder(x) 
        
    @property
    def parameters(self):
        """ A convenience function for getting all the parameters of our model.
        
        This can be accessed as an attribute, via `model.parameters` 
        
        Returns
        -------
        Tuple[Tensor, ...]
            A tuple containing all of the learnable parameters for our model """
        
        return self.encoder.parameters

In [ ]:
def accuracy(predictions, truth):
    """
    Returns the mean classification accuracy for a batch of predictions.
    
    Parameters
    ----------
    predictions : Union[numpy.ndarray, mg.Tensor], shape=(N, 2)
        The scores for 2 classes, for a batch of N data points
        
    truth : numpy.ndarray, shape=(N,)
        The true labels for each datum in the batch: each label is an
        integer in [0, 1]
    
    Returns
    -------
    float
    """
   
    if isinstance(predictions, mg.Tensor):
        predictions = predictions.data
    return np.mean(np.argmax(predictions, axis=1) == truth)

In [ ]:
model = LinearEncoder(512, 50)